# AI Agent

This notebook is to test the basics related to the AI Agent

In [14]:
from pydantic_ai import Agent, ImageUrl
from pydantic_ai.models.google import GoogleModel, GoogleModelSettings
from pydantic_ai.providers.google import GoogleProvider

import sys

sys.path.append("..")

from agent.config import AgentConfig

## Set up a basic AI Agent

In [15]:
agent_config = AgentConfig()

# To load the Gemini API key from GCP Secret Manager
agent_config.load_gemini_api_key()

In [16]:
provider = GoogleProvider(api_key=agent_config.GEMINI_API_KEY.get_secret_value())
model = GoogleModel(model_name=agent_config.GEMINI_MODEL_NAME, provider=provider)
model_settings = GoogleModelSettings(temperature=agent_config.MODEL_TEMPERATURE)

agent = Agent(model=model, model_settings=model_settings)

## Execute a simple agent request

For more info, use the [pydantic-AI documentation](https://ai.pydantic.dev/agents/#introduction)

In [4]:
response = await agent.run("Whats your name?")

In [5]:
response.output

'I do not have a name. I am a large language model, trained by Google.'

In [6]:
# Retrieves a list of all messages (ModelMessage objects) exchanged during the conversation

response.all_messages()

[ModelRequest(parts=[UserPromptPart(content='Whats your name?', timestamp=datetime.datetime(2025, 10, 17, 21, 56, 12, 432920, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content='I do not have a name. I am a large language model, trained by Google.')], usage=RequestUsage(input_tokens=5, output_tokens=236, details={'thoughts_tokens': 218, 'text_prompt_tokens': 5}), model_name='gemini-2.5-pro', timestamp=datetime.datetime(2025, 10, 17, 21, 56, 15, 195230, tzinfo=datetime.timezone.utc), provider_name='google-gla', provider_details={'finish_reason': 'STOP'}, provider_response_id='f7vyaLDqBqmvqtsP0qDjmAs', finish_reason='stop')]

## Keep the chat history

In [11]:
history = [] # List of ModelMessage objects

user_input = "What is nuclear energy?"
response = await agent.run(user_input, message_history=history)
history = response.all_messages()
print("Agent:", response.output)

user_input = "What are its pros and cons?"
response = await agent.run(user_input, message_history=history)
history = response.all_messages()
print("Agent:", response.output)

last_answer = await agent.run("Summarize our conversation in a few sentences", message_history=response.all_messages())

last_answer.output

Agent: Of course! Here is a comprehensive explanation of nuclear energy, broken down into simple terms.

### **What is Nuclear Energy? The Simple Answer**

At its core, **nuclear energy is the energy stored in the nucleus (the core) of an atom.**

Atoms are the tiny particles that make up every object in the universe. This energy can be released in two main ways:
1.  **Nuclear Fission:** Splitting an atom apart. (This is how all current nuclear power plants work).
2.  **Nuclear Fusion:** Fusing two atoms together. (This is how the sun creates energy, and it's a technology we are trying to develop on Earth).

Because the forces holding an atom's nucleus together are incredibly strong, releasing that energy creates a tremendous amount of heat.

---

### **How Nuclear Power Plants Work (Nuclear Fission)**

Think of a nuclear power plant as a giant, high-tech kettle. Its main job is to boil water to create steam, which then turns a turbine to generate electricity. The only thing that makes

'Of course. Here is a summary of our conversation:\n\nNuclear energy is the power released from splitting atoms (a process called fission) to generate immense heat, which is then used to create electricity. Its greatest advantage is producing a massive amount of reliable, carbon-free power, making it a powerful tool against climate change. However, it is highly controversial due to major challenges like the long-term storage of radioactive waste, the risk of catastrophic accidents, and the enormous cost of building new plants.'

In [13]:
last_answer.all_messages()

[ModelRequest(parts=[UserPromptPart(content='What is nuclear energy?', timestamp=datetime.datetime(2025, 10, 18, 2, 33, 9, 774320, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[TextPart(content="Of course! Here is a comprehensive explanation of nuclear energy, broken down into simple terms.\n\n### **What is Nuclear Energy? The Simple Answer**\n\nAt its core, **nuclear energy is the energy stored in the nucleus (the core) of an atom.**\n\nAtoms are the tiny particles that make up every object in the universe. This energy can be released in two main ways:\n1.  **Nuclear Fission:** Splitting an atom apart. (This is how all current nuclear power plants work).\n2.  **Nuclear Fusion:** Fusing two atoms together. (This is how the sun creates energy, and it's a technology we are trying to develop on Earth).\n\nBecause the forces holding an atom's nucleus together are incredibly strong, releasing that energy creates a tremendous amount of heat.\n\n---\n\n### **How Nuclear Power Plants

## Adding images to the input

In [23]:
response = await agent.run([
    "What image is this?", 
    ImageUrl(
        url="https://th.bing.com/th/id/R.56465a807f77d56fc9a6fd4d0a94f1c2?rik=9U3hFwTXDtqJOg&riu=http%3a%2f%2fwallpapercave.com%2fwp%2fwc1674567.jpg&ehk=t0bjDuQ0Mc34PFHBiFLjjk7Yg3ma0HaksetqDISHEX0%3d&risl=&pid=ImgRaw&r=0")
    ])

response.output

'Based on the image provided, this is a promotional-style wallpaper or poster of the **Real Madrid football club from the 2014-2015 season**.\n\nHere are the key details that identify the image:\n\n*   **The Team:** It features the star-studded lineup of Real Madrid from that era.\n*   **The Occasion:** The image celebrates two major achievements:\n    *   **La Décima:** The badge in the top left corner with the number "10" signifies their 10th UEFA Champions League title, won in the 2013-2014 season.\n    *   **FIFA Club World Cup Champions:** The badge in the top right corner is the official FIFA World Champions patch, which Real Madrid earned by winning the Club World Cup in December 2014. This specific patch places the image firmly in the 2014-2015 season.\n*   **The Kit:** The players are wearing the 2014-2015 home kit, which was white with black Adidas stripes. The stylized dragons in the background were a design motif used by the club that season, notably on their black third ki

## Agent Tools

In this section of the notebook, some tools that the agent will use are tested.

In [3]:
import sys

sys.path.append("..")

from agent.tools.text_to_speech import text_to_speech
from agent.tools.cloud_storage import upload_text_to_gcs
from agent.tools.image_generation import generate_images

from agent.tools.tool_schemas import ImaGenRequest

### Text To Speech Tool

In [3]:
audio_url = text_to_speech(
    title="AudioTest",
    text ="Hi, welcome to Pydantic AI Agents!",
    mode="single"
)

2025-10-20 01:37:53.872 | INFO     | agent.tools.text_to_speech:text_to_speech:148 - Using Text to Speech Tool...
2025-10-20 01:37:53.874 | DEBUG    | agent.tools.text_to_speech:text_to_speech:174 - title ='audiotest'
2025-10-20 01:37:53.875 | DEBUG    | agent.tools.text_to_speech:text_to_speech:175 - text ='Hi, welcome to Pydantic AI Agents!'
2025-10-20 01:37:53.875 | DEBUG    | agent.tools.text_to_speech:text_to_speech:176 - gcs_path ='audios'
2025-10-20 01:37:53.876 | DEBUG    | agent.tools.text_to_speech:text_to_speech:177 - full_gcs_path ='audios/audiotest.wav'
2025-10-20 01:37:53.877 | DEBUG    | agent.tools.text_to_speech:text_to_speech:179 - Creating audio...
2025-10-20 01:37:53.878 | INFO     | agent.tools.text_to_speech:single_speaker_tts_audio:54 - Generating single-speaker TTS audio...
2025-10-20 01:37:57.511 | DEBUG    | agent.tools.text_to_speech:single_speaker_tts_audio:75 - Converting PCM to WAV bytes...
2025-10-20 01:37:57.512 | DEBUG    | agent.tools.text_to_speech:te

In [4]:
audio_url = text_to_speech(
    title="Multi speaker speech",
    text="Wendy: Hi, Emma!\nEmma: Hi, Wendy!",
    mode="multi",
)

2025-10-20 01:39:59.070 | INFO     | agent.tools.text_to_speech:text_to_speech:148 - Using Text to Speech Tool...
2025-10-20 01:39:59.073 | DEBUG    | agent.tools.text_to_speech:text_to_speech:174 - title ='multi_speaker_speech'
2025-10-20 01:39:59.074 | DEBUG    | agent.tools.text_to_speech:text_to_speech:175 - text ='Wendy: Hi, Emma!\nEmma: Hi, Wendy!'
2025-10-20 01:39:59.075 | DEBUG    | agent.tools.text_to_speech:text_to_speech:176 - gcs_path ='audios'
2025-10-20 01:39:59.075 | DEBUG    | agent.tools.text_to_speech:text_to_speech:177 - full_gcs_path ='audios/multi_speaker_speech.wav'
2025-10-20 01:39:59.076 | DEBUG    | agent.tools.text_to_speech:text_to_speech:179 - Creating audio...
2025-10-20 01:39:59.078 | INFO     | agent.tools.text_to_speech:multi_speaker_tts_audio:91 - Generating multi-speaker TTS audio...
2025-10-20 01:40:01.253 | DEBUG    | agent.tools.text_to_speech:multi_speaker_tts_audio:125 - Converting PCM to WAV bytes...
2025-10-20 01:40:01.254 | DEBUG    | agent.too

### Google Cloud Storage Tools

In [ ]:
upload_text_to_gcs(
    text="Hi, world!",
    blob_name="test.txt"
)

2025-10-20 02:00:48.984 | INFO     | utils.gcp.gcs:upload_bytes:184 - Bytes data successfully stored in GCS bucket


## Image Generation

### Text To Image

In [6]:
prompts = [
    ImaGenRequest(prompt="A tree inside an apple in an animated drawing", image_name="tree_inside_apple"),
    ImaGenRequest(prompt="A spider in his web", image_name="spider_in_web"),
]

images_generated = await generate_images(prompts)

2025-10-21 19:16:10.443 | INFO     | agent.tools.image_generation:generate_images:71 - Using Image Generation Tool...
2025-10-21 19:16:10.446 | INFO     | agent.tools.image_generation:generate_images:82 - Generating 2 images...
2025-10-21 19:16:10.448 | DEBUG    | agent.tools.image_generation:generate_images:84 - image_requests = [ImaGenRequest(prompt='A tree inside an apple in an animated drawing', image_name='tree_inside_apple'), ImaGenRequest(prompt='A spider in his web', image_name='spider_in_web')]
2025-10-21 19:16:10.448 | DEBUG    | agent.tools.image_generation:generate_images:88 - Preparing generation requests
2025-10-21 19:16:10.449 | DEBUG    | agent.tools.image_generation:generate_images:95 - Launching concurrent generation
2025-10-21 19:16:10.450 | DEBUG    | agent.tools.image_generation:generate_image:33 - general_image_name = 'tree_inside_apple'
2025-10-21 19:16:10.450 | DEBUG    | agent.tools.image_generation:generate_image:34 - Input prompt = 'A tree inside an apple in 

In [7]:
images_link = [image.public_url for image in images_generated]
images_link

['https://storage.googleapis.com/ai-hub-sharepoint/images/tree_inside_apple.png',
 'https://storage.googleapis.com/ai-hub-sharepoint/images/spider_in_web.png']